In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import learning.min_regret_strategy_pb2
import experimental.pokerbots.bin_centers_pb2
import numpy as np
import experimental.pokerbots.pokerbot as pokerbot
import experimental.pokerbots.pokerbots_scrape as pbs
import experimental.pokerbots.hand_evaluator_python as hep
import pandas as pd
import os

import importlib
importlib.reload(pokerbot)

In [ ]:
CLUSTER_PATH = '/home/erick/code/robot/experimental/pokerbots/cluster_hands'
flop_data = pd.read_pickle(os.path.join(CLUSTER_PATH, 'flop_data.p'))
turn_data = pd.read_pickle(os.path.join(CLUSTER_PATH, 'turn_data.p'))
river_data = pd.read_pickle(os.path.join(CLUSTER_PATH, 'river_data.p'))

In [ ]:
strategy_file = '/home/erick/code/robot/experimental/pokerbots/pokerbot_checkpoint_031070000.pb'
with open(strategy_file, 'rb') as file_in:
    strategy = learning.min_regret_strategy_pb2.MinRegretStrategy()
    strategy.ParseFromString(file_in.read())
bin_centers_file = '/home/erick/code/robot/experimental/pokerbots/bin_centers.pb'
with open(bin_centers_file, 'rb') as file_in:
    per_turn_bin_centers = experimental.pokerbots.bin_centers_pb2.PerTurnBinCenters()
    per_turn_bin_centers.ParseFromString(file_in.read())

In [ ]:
PAGE_LIMIT = 1
session_value = pbs.load_session_value()
rows = pbs.pull_rows(PAGE_LIMIT, session_value)

In [ ]:
game_log = pbs.get_game_log(rows.loc[2, 'game_log'], session_value)

In [ ]:
large_hands = list(filter(lambda game : abs(game.A_delta) > 50, game_log))

In [ ]:
large_hands = sorted(large_hands, key = lambda x: x.A_delta)

In [ ]:
def print_game(game_round):
    for board, actions in zip(game_round.board_states, game_round.round_actions):
        if board:
            print('Board:', board)
        for a in actions:
            print(a)
    

In [ ]:
def find_infoset(infoset_id, strategy=strategy):
    return list(filter(lambda infoset: infoset.id_num == infoset_id, strategy.infoset_counts))[0]

In [ ]:
print_game(large_hands[0])

In [ ]:
actions = ['Check']
betting_round = 3
hand = ['Jd', 'Ts']
board_cards = ['2d', 'Qs', 'Ad', '8h', '3h', '6c']
ids = pokerbot.compute_infoset_id(actions, betting_round, hand, board_cards, per_turn_bin_centers)
print(hex(ids[0]))
infoset = find_infoset(ids[0])
print(infoset)
strategy = np.array(infoset.strategy_sum) / np.sum(infoset.strategy_sum)
print(strategy)

In [ ]:
bucket_hands = river_data.loc[river_data['cluster_idx'] == (ids[0] & 0xFF)]

In [ ]:
bucket_hands.mean(numeric_only=True)

In [ ]:
bucket_hands

In [ ]:
result = hep.evaluate_strength_potential(''.join(hand), ''.join(board_cards), 2, atimeout_s=0.02)

In [ ]:
print(result.strength, result.negative_potential, result.positive_potential)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
_ = plt.hist(bucket_hands['strength'],bins=100)
plt.subplot(132)
_ = plt.hist(bucket_hands['neg_pot'],bins=100)
plt.subplot(133)
_ = plt.hist(bucket_hands['pos_pot'],bins=100)
plt.tight_layout()